In [1]:
import torch
from transformers import AutoConfig, AutoModel
from transformers import RobertaTokenizer, RobertaForMaskedLM, RobertaForCausalLM, RobertaForSequenceClassification
from datasets import Dataset, load_dataset

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# tokenizer = ByteLevelBPETokenizer()

# model = AutoModel.from_pretrained("chrishuber/roberta-retrained-mlni")
# model = RobertaForMaskedLM.from_pretrained("roberta-base")

model = RobertaForSequenceClassification.from_pretrained("roberta-base")
# model = RobertaForCausalLM.from_pretrained("roberta-base")
# model.to('cuda');

# model = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
# model = TransformerModel('roberta', 'roberta-base', args=({'fp16': False}))
# model.load_state_dict(torch.load('./roberta-mnli-retrained/pytorch_model.bin'))
model.eval()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [4]:
# from tokenizers import ByteLevelBPETokenizer

# # Initialize a tokenizer
# tokenizer = ByteLevelBPETokenizer(lowercase=True)

# # Customize training
# tokenizer.train(files=["./multinli_1.0/multinli_1.0/multinli_1.0_dev_matched.txt"], vocab_size=8192, min_frequency=2,
#                 show_progress=True,
#                 special_tokens=[
#                                 "<s>",
#                                 "<pad>",
#                                 "</s>",
#                                 "<unk>",
#                                 "<mask>",
# ])
# #Save the Tokenizer to disk
# tokenizer.save_model("./tokenizers")

In [5]:
# tokenizer = ByteLevelBPETokenizer(
#     './tokenizers/vocab.json',
#     './tokenizers/merges.txt'
# )

In [2]:
from transformers import DataCollatorForLanguageModeling, DataCollatorForTokenClassification

data_collator = DataCollatorForLanguageModeling(
     tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# data_collater = DataCollatorForTokenClassification(
#     tokenizer=tokenizer
# )

def tokenize_function(batch):
    # tokens = tokenizer(batch["sentence1"], batch["sentence2"], padding="max_length", truncation=True)
    tokens = tokenizer(["sentence1", "sentence2"], padding="max_length", truncation=True)
    return tokens

In [7]:
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
# print(output)

In [8]:
# raw_datasets["test"].to_pandas().head()

In [3]:
raw_datasets = load_dataset("chrishuber/kaggle_mnli")

# tokenized_test_dataset.to_pandas().tail()

Using custom data configuration chrishuber--kaggle_mnli-df90bb2b9c35e99b
Reusing dataset json (C:\Users\chris\.cache\huggingface\datasets\json\chrishuber--kaggle_mnli-df90bb2b9c35e99b\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
test_df = raw_datasets['test']
test_df.shape
test_df = test_df.to_pandas()
test_df.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[hidden, hidden, hidden, hidden, hidden]",slate,hidden,9847,9847,That which binds together Chinese.,( ( That ( which ( binds ( together Chinese ) ...,(ROOT (FRAG (NP (NP (DT That)) (SBAR (WHNP (WD...,This is a shared value among Chinese people.,( This ( ( is ( ( a ( shared value ) ) ( among...,(ROOT (S (NP (DT This)) (VP (VBZ is) (NP (NP (...
1,"[hidden, hidden, hidden, hidden, hidden]",government,hidden,9848,9848,The actual length of an individual worker's H-...,( ( ( The ( actual length ) ) ( of ( ( an ( in...,(ROOT (S (NP (NP (DT The) (JJ actual) (NN leng...,The location of the employer effects the lengt...,( ( ( The location ) ( of ( the employer ) ) )...,(ROOT (S (NP (NP (DT The) (NN location)) (PP (...
2,"[hidden, hidden, hidden, hidden, hidden]",fiction,hidden,9849,9849,Every man I put down left me empty.,( ( ( Every man ) ( I ( put down ) ) ) ( ( lef...,(ROOT (S (NP (NP (DT Every) (NN man)) (SBAR (S...,I felt empty after every man I put down.,( I ( ( ( felt ( empty ( after ( every man ) )...,(ROOT (S (NP (PRP I)) (VP (VBD felt) (ADJP (JJ...
3,"[hidden, hidden, hidden, hidden, hidden]",telephone,hidden,9850,9850,and uh i really think that if uh like after se...,( and ( ( uh i ) ( really ( think ( ( that if ...,(ROOT (FRAG (CC and) (NP (NP (FW uh) (FW i)) (...,Women wouldn't have gone to work after the sec...,( Women ( ( ( would n't ) ( have ( gone ( to (...,(ROOT (S (NP (NNP Women)) (VP (MD would) (RB n...
4,"[hidden, hidden, hidden, hidden, hidden]",telephone,hidden,9851,9851,yep yeah yeah it was i ended up going into ban...,( ( yep yeah ) ( yeah ( ( ( ( it ( ( was i ) (...,(ROOT (S (NP (NN yep) (NN yeah)) (VP (VBP yeah...,I have no idea what bankruptcy is like.,( I ( ( have ( ( no idea ) ( what ( bankruptcy...,(ROOT (S (NP (PRP I)) (VP (VBP have) (NP (NP (...


In [5]:
tokenized_test_dataset = raw_datasets["test"].map(tokenize_function)

Loading cached processed dataset at C:\Users\chris\.cache\huggingface\datasets\json\chrishuber--kaggle_mnli-df90bb2b9c35e99b\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-c76dbe2b101d4dbf.arrow


In [12]:
# from torch.utils.data import DataLoader
# test_dataloader = DataLoader(tokenized_test_dataset)

# tokenized_df = tokenized_test_dataset.to_pandas()
# tokenized_df.drop(["sentence1", "sentence2", "sentence1_binary_parse", "sentence1_parse", "sentence2_binary_parse", "sentence2_parse", "annotator_labels", "genre", "pairID", "promptID"], axis=1, inplace=True)
# tokenized_df.rename(columns={"gold_label": "labels"}, inplace=True)
# tokenized_df.head()

# tokenized_test_dataset = Dataset.from_pandas(tokenized_df)
# tokenized_test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [6]:
# here i am getting output for a single tuple containing two sentences
tokens = tokenizer.encode((('Roberta is a heavily optimized version of BERT', 'Roberta is not very optimized')))

output = model(torch.tensor([tokens]))
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2374, -0.0192]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [22]:
# passing more results in an error: how do I run all the sentences?
tokens = tokenizer.encode((('Roberta is a heavily optimized version of BERT', 'Roberta is not very optimized'),('I am happy.', 'I am not happy.')))
output = model(torch.tensor([tokens]))
print(output)

ValueError: Input (('Roberta is a heavily optimized version of BERT', 'Roberta is not very optimized'), ('I am happy.', 'I am not happy.')) is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
import numpy as np

preds = []

# preds = model(input_ids=b_input_ids, token_type_ids=b_token_type_ids, attention_mask=b_input_mask)
for i, item in enumerate(output.logits):
    print(i, item)
    if i == 0:
        print(item)
        preds = item
    else:
        # Sum the matrices
        the_preds = item + preds
        preds.append(the_preds)

preds
    
# Average the predictions
avg_preds = preds/(len(output))

test_preds = np.argmax(avg_preds.detach().numpy(), axis=1)

In [147]:
len(test_preds)

4

In [117]:
tokenized_test = Dataset.from_pandas(tokenized_df)
tokenized_test.set_format("torch")
tokenized_test.features

{'labels': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [72]:
# preds = trainer.predict(tokenized_test_dataset)
# preds = model(tokenized_test)

from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
# outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]
pipe(tokenized_test_dataset)

The model 'RobertaModel' is not supported for . Supported models are ['BartForSequenceClassification', 'YosoForSequenceClassification', 'NystromformerForSequenceClassification', 'PLBartForSequenceClassification', 'PerceiverForSequenceClassification', 'QDQBertForSequenceClassification', 'FNetForSequenceClassification', 'GPTJForSequenceClassification', 'LayoutLMv2ForSequenceClassification', 'RemBertForSequenceClassification', 'CanineForSequenceClassification', 'RoFormerForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BigBirdForSequenceClassification', 'ConvBertForSequenceClassification', 'LEDForSequenceClassification', 'DistilBertForSequenceClassification', 'AlbertForSequenceClassification', 'CamembertForSequenceClassification', 'XLMRobertaXLForSequenceClassification', 'XLMRobertaForSequenceClassification', 'MBartForSequenceClassification', 'BartForSequenceClassification', 'LongformerForSequenceClassification', 'RobertaForSequenceClassification', 'Data2VecTextForSe

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).